# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
import numpy as np

try:
    data = pd.read_csv('C:/Users/datasets/data.csv')  
except:
    data = pd.read_csv('/datasets/data.csv')

In [2]:
#изучим первые 10 строк таблицы

data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
#получим общую информацию о датафрейме

data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [4]:
#получим основную информацию по таблице

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
#определим долю пропущенных значений каждого столбца датафрейма
data.isna().mean()

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

**Вывод**

В таблице 'data.csv' представлен практически полный набор данных, за исключением двух столбцов: 'days_employed' и 'total_income'. Также выделяется столбец 'education', данные в котором введены с использованием различных регистров. 
Помимо этого видно, что категориальные переменные 'education_id', 'family_status_id' и 'debt' неверно записаны как количественные переменные: им присвоен тип int (используется для представления целых чисел).

## Шаг 2. Предобработка данных

### Обработка пропусков

In [6]:
import warnings
warnings.simplefilter("ignore")


#проверим какие данные были пропущены в исходном датафрейме
data_isna = data[data['days_employed'].isna()]

data_isna

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [7]:
#пропущенные данные по значениям "income_type"
data_isna['income_type'].value_counts().sum()

2174

<div style="border: 15px; border: 2px solid blue; padding: 15px;">
<b> Комментарий студента:</b>
<hr>
    Проведем оценку входных данных. Выясним почему в столбце 'days_employed' имеются отрицательные значения и аномально высокие значения.

</div>

In [8]:
#для начала проведем оценку данных для случая, когда стаж в днях больше нуля

days_employed_below_zero = data[data['days_employed']>0]
days_employed_below_zero

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем


In [9]:
#посмотрим какие уникальные категории встречаются в полученном датафрейме
days_employed_below_zero['income_type'].value_counts()

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

In [10]:
#проверим сколько значений содержалось в исходном датафрейме с категорией "пенсионеры"
data[data['income_type'] == 'пенсионер']['days_employed'].count()

3443

<div style="border: 15px; border: 2px solid blue; padding: 15px;">
<b> Комментарий студента:</b>
<hr>
   Как показывает анализ выше, наибольшее число данных из датафрейма "days_employed_below_zero" относится к категории людей "пенсионеры". Это 100% исходного датафрейма для категории "пенсионеры". Так что будем разбираться что тут не так))
</div>

In [11]:
#оценим медианное значение стажа в днях для нового датафрейма
days_employed_below_zero['days_employed'].median()

365213.30626573117

<div style="border: 15px; border: 2px solid blue; padding: 15px;">
<b> Комментарий студента:</b>
<hr>
  Полученное значение является аномально большим. Попробуем проверить, возможно ли такое, что по какой-то ошибке данные по данной группе были выгружены в часах, а не в днях.
</div>

In [12]:
#переведем значение столбца 'days_employed' в года

days_employed_below_zero['years_employed'] = days_employed_below_zero['days_employed']/(24*365)

days_employed_below_zero

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,38.843159
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля,45.694194
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью,38.647483
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости,41.500969
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью,38.308410
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу,38.687770
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость,44.120744
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью,41.342586
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем,42.693574


In [13]:
#вычислим медианный возраст людей из нашего датафрейма

dob_years_avg = days_employed_below_zero['dob_years'].median()
dob_years_avg

60.0

In [14]:
#вычислим медианный стаж, полученный для данного датафрейма в годах

years_employed_avg = days_employed_below_zero['years_employed'].median()
years_employed_avg

41.691016697001274

In [15]:
#вычислим предполагаемую дату начала работы людей из данной группы

start_employed = dob_years_avg-years_employed_avg
start_employed

18.308983302998726

<div style="border: 15px; border: 2px solid blue; padding: 15px;">
<b> Комментарий студента:</b>
<hr>
  Полученное значение start_employed подтвердило предположение о том, что данные по группе "пенсионеры" были выгружены некорректно ввиду какой-либо ошибки. 
</div>

In [16]:
#посчитаем значение стажа в днях для категории "пенсионеры" в исходном датафрейме

data['days_employed'] = np.where(data['days_employed']>0, data['days_employed']/24, data['days_employed'])

data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,14330.725172,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [17]:
#проверим теперь значение стажа в днях датафрейма с категорией "пенсионеры"
data[data['income_type'] == 'пенсионер']['days_employed'].median()

15217.221094405466

<div style="border: 15px; border: 2px solid blue; padding: 15px;">
<b> Комментарий студента:</b>
<hr>
  Теперь проанализируем отрицательные значения стоблца "days_employed" исходного датафрейма.
</div>

In [18]:
# создадим новый датафрейм 'days_employed_abowe_zero'
#и узнаем медианное значение стажа для датафрейма days_employed_abowe_zero

days_employed_abowe_zero = data[data['days_employed']<=0]

days_employed_med = days_employed_abowe_zero['days_employed'].median()

days_employed_med

-1630.0193809778216

In [19]:
#посмотрим какие уникальные категории встречаются в полученном датафрейме
days_employed_abowe_zero['income_type'].value_counts()

сотрудник          10014
компаньон           4577
госслужащий         1312
предприниматель        1
в декрете              1
студент                1
Name: income_type, dtype: int64

In [20]:
#также узнаем средний возраст людей датафрейма 'days_employed_abowe_zero'

dob_years_med = days_employed_abowe_zero['dob_years'].median()

dob_years_med

39.0

In [21]:
#узнаем стаж в годах людей датафрейма 'days_employed_abowe_zero'

avg = days_employed_med/365

avg

-4.465806523226909

<div style="border: 15px; border: 2px solid blue; padding: 15px;">
<b> Комментарий студента:</b>
<hr>
  Полученные выше результаты говорят о том, что данные вполне реальны и ошибка закралась в данные при выгрузке. Освободим наши данные от знака минус в исходном датафрейме.
</div>

In [22]:
#сделаем так, чтобы все значения столбца "days_employed" исходного датафрейма стали положительными
data['days_employed'] = abs(data['days_employed'])

data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,14330.725172,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


<div style="border: 15px; border: 2px solid blue; padding: 15px;">
<b> Комментарий студента:</b>
<hr>
  Далее заменим пропущенные значения столбца 'days_employed' и 'total_income' медианными значениями в зависимости от значения в столбце 'income_type'.
</div>

In [23]:
#произведем замену пустых значений столбцов 'days_employed' и 'total_income' медианными значениями 
#в зависимости от значения в столбце 'income_type'.

for column in ['days_employed', 'total_income']:
    data[column] = data[column].fillna(data.groupby('income_type')[column].transform('median')) 

data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,14330.725172,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [24]:
#проверим датафрейм на наличие пропущенных данных
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

<div style="border: 15px; border: 2px solid blue; padding: 15px;">
<b> Комментарий студента:</b>
<hr>
  Как можно заметить, в датафрейме не осталось строк с пропущенными значениями. Теперь перейдем к исправлению данных в столбцах 'children' и 'gender'.
</div>

In [25]:
#в столбце 'children' имеются отрицательное значение и аномально большие. 

data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [26]:
#Заменим значения стоблца 'children' у даннных строк на ноль.

data['children'] = np.where(((data['children'] >= 20) | (data['children'] < 0)), 0, data['children'])

In [27]:
#проверим изменения
data['children'].value_counts()

0    14272
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

In [28]:
#также в нашем датфрейме есть непонятное значение в столбце 'gender'

data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [29]:
#полностью удалим данную строку
gender_xna = data['gender'] == 'XNA'

data['gender'] = data['gender'][~gender_xna]
data['gender'].value_counts()

F    14236
M     7288
Name: gender, dtype: int64

**Вывод**

В данном разделе была проведена обработка 2174 пропусков. Пропуски были заменены на медианные значения в зависимости от групп. Также была проведена работа по приведению данных в надлежащий вид. Обработаны отрицательные и аномальные значения столбцов "days_employed" и "children", также обработан столбец "gender".

### Замена типа данных

<div style="border: 15px; border: 2px solid blue; padding: 15px;">
<b> Комментарий студента:</b>
<hr>
    Видно, что категориальные переменные 'education_id', 'family_status_id' и 'debt' неверно записаны как количественные переменные: им присвоен тип int (используется для представления целых чисел). Преобразуем категориальные переменные 'education_id', 'family_status_id' и 'debt' в тип данных 'object'. Также преобразуем тип переменных 'days_employed' и 'total_income' из вещественного типа в целочисленный.       
</div>


In [30]:
#преобразуем категориальные переменные 'education_id', 'family_status_id' и 'debt' в тип данных 'object'
for i in ['education_id', 'family_status_id', 'debt']:
    if i in data.columns:
        data[i]=data[i].astype('object')

In [31]:
#преобразуем вещественные переменные 'days_employed' и 'total_income' в тип данных 'int'
for i in ['days_employed', 'total_income' ]:
    if i in data.columns:
        data[i]=data[i].astype('int')

In [32]:
#проверим внесенные изменения
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null object
family_status       21525 non-null object
family_status_id    21525 non-null object
gender              21524 non-null object
income_type         21525 non-null object
debt                21525 non-null object
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(4), object(8)
memory usage: 2.0+ MB


**Вывод**

В данном разделе неверно записанным переменным 'days_employed', 'total_income', 'education_id', 'family_status_id' и 'debt' был присвоен верный тип. Теперь все переменные в нашем датафрейме обладают соответствующим типом данных.

### Обработка дубликатов

In [33]:
# для начала изменим регистр в столбце 'education'

data['education'] = data['education'].str.lower()

In [34]:
#посчитаем количество строк-дубликатов в датафрейме
data.duplicated().sum()

71

In [35]:
#узнаем размер исходного датафрейма

shape_table = data.shape

shape_table

(21525, 12)

In [36]:
#выведем первые 10 строк-дубликатов

duplicated_data = data[data.duplicated()]
duplicated_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,1574,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи
3290,0,15217,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
4182,1,1574,34,высшее,0,гражданский брак,1,F,сотрудник,0,142594,свадьба
4851,0,15217,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба
5557,0,15217,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
6312,0,1574,30,среднее,1,женат / замужем,0,M,сотрудник,0,142594,строительство жилой недвижимости
7808,0,15217,57,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
7921,0,15217,64,высшее,0,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
7938,0,15217,71,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
8583,0,15217,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,118514,дополнительное образование


In [37]:
#удалим строки-дубликаты из датафрейма

data = data.drop_duplicates().reset_index(drop=True)

In [38]:
#узнаем размер датафрейма после удаления дубликатов

shape_table_update = data.shape

shape_table_update

(21454, 12)

**Вывод**

В исходном датафрейме содержалась 71 строка-дубликат, после удаления данных строк размер датафрейма уменьшился.

### Лемматизация

<div style="border: 15px; border: 2px solid blue; padding: 15px;">
<b> Комментарий студента:</b>
<hr>
    Чтобы выделить леммы в целях кредита необходимо провести процесс лемматизации столблца data['purpose']. Для этой цели используют библиотеку Pymystem3. А чтобы пробежаться по строкам датафрейма и применить к ним функцию лемматизации необходимо написать дополнительную функцию и применить ее к датафрейму с помощью метода .apply().

</div>

In [1]:
from pymystem3 import Mystem
m = Mystem() 

def lemmas(row):
    
    purpose = row['purpose']
    
    lemmatized = m.lemmatize(purpose)
    
    return lemmatized

data['lemmatized_purpose'] = data.apply(lemmas, axis=1)

data.head(10)


Installing mystem to C:\Users\IshkulovIM/.local/bin\mystem.exe from http://download.cdn.yandex.net/mystem/mystem-3.1-win-64bit.zip


PermissionError: [WinError 32] Процесс не может получить доступ к файлу, так как этот файл занят другим процессом: 'C:\\Users\\ISHKUL~1\\AppData\\Local\\Temp\\tmp0qnmbf1o'

In [40]:
#посчитаем на какие цели в основном берут кредиты

print(data['lemmatized_purpose'].value_counts())

[автомобиль, \n]                                          972
[свадьба, \n]                                             791
[на,  , проведение,  , свадьба, \n]                       768
[сыграть,  , свадьба, \n]                                 765
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           661
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 651
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            624
[покупка

<div style="border: 15px; border: 2px solid blue; padding: 15px;">
<b> Комментарий студента:</b>
<hr>
    Далее необходимо почистить наши цели от лишних слов. Вера, подскажите, как можно не прибегая к методам ниже, не вручную и не пробегая глазами, подсчитать самые популярные слова с учетом того, что они представлены в списках на каждой строке?
</div>

In [41]:
#сформируем список с наиболее популярными категориями

purpose_word = ['автомобиль', 'жилье', 'образование', 'свадьба', 'недвижимость']

# далее заменим значения столбца 'lemmatized_purpose' на ключевые слова из списка 'purpose_word'
data_1 = data.explode('lemmatized_purpose')

data_2 = data_1[data_1.lemmatized_purpose.isin(purpose_word)] 


In [42]:
#оценим на какие цели чаще брали кредиты
print(data_2['lemmatized_purpose'].value_counts())

недвижимость    6351
жилье           4460
автомобиль      4306
образование     4013
свадьба         2324
Name: lemmatized_purpose, dtype: int64


In [43]:
#в полученных целях кредита есть схожие категории, такие как "недвижимость" и "жилье". Объединим данные категории в категорию "недвижимость".

data_2['lemmatized_purpose'] = data_2['lemmatized_purpose'].replace(to_replace=r'жилье', value='недвижимость')


data_2

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmatized_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,недвижимость
21450,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль
21451,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость
21452,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль


In [44]:
#проверим изменения
print(data_2['lemmatized_purpose'].value_counts())

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: lemmatized_purpose, dtype: int64


<div style="border: 15px; border: 2px solid blue; padding: 15px;">
<b> Комментарий студента:</b>
<hr>
    По результатам анализа выше можно заметить, что наиболее часто люди берут кредит для покупки недвижимости. Теперь необходимо оценить как влияет цель кредита на возврат кредита в срок.
</div>

In [45]:
#посчитаем кол-во людей с задолженностью в каждой группе и на какие цели чаще всего берут кредиты

purpose_debt = data_2.groupby('lemmatized_purpose').agg({'debt':['sum', 'count']})

purpose_debt

debt       
                    sum  count
lemmatized_purpose            
автомобиль          403   4306
недвижимость        782  10811
образование         370   4013
свадьба             186   2324

In [46]:
#вычислим отношение количества задолженностей в зависимости от целей кредита к общему числу людей,
#которые брали кредит на те или иные цели

purpose_debt_avg = purpose_debt['debt']['sum']/purpose_debt['debt']['count']
purpose_debt_avg.sort_values(ascending=False)

lemmatized_purpose
автомобиль      0.093590
образование     0.092200
свадьба         0.080034
недвижимость    0.072334
dtype: float64

In [47]:
#посчитаем медианный ежемесячный доход в зависимости от цели кредита

purpose_income = data_2.groupby('lemmatized_purpose')['total_income'].median().sort_values(ascending=False)
purpose_income

lemmatized_purpose
недвижимость    142895
свадьба         142594
образование     142594
автомобиль      142594
Name: total_income, dtype: int64

**Вывод**

Был проведен анализ целей кредита с помощью лемматизации. Также было установлено как влияют цели кредита на возврат кредита в срок. По результатам проведенной работы можно сказать, что наиболее часто имеют задолженность люди, которые берут кредит на автомобили и образование, это связано с тем, что ежемесячный доход у этих групп людей ниже среднего. При этом меньше всего людей имеют задолженность по кредиту на недвижимость, это объясняется более высоким ежемесячным доходом.

### Категоризация данных

<div style="border: 15px; border: 2px solid blue; padding: 15px;">
<b> Комментарий студента:</b>
<hr>
    Для начала ответим на вопрос есть ли зависимость между наличием детей и возвратом кредита в срок? Для этого поделим выборку на тех, у кого есть дети и у кого их нет.
</div>

In [48]:
#сначала создадим новую категорию, в которой отразим наличие ребенка, либо его отсутствие.

def child_ex(row):
    
    child_exist = row['children']
    
    if child_exist > 0:
        return 'есть ребенок'
    
    return 'нет детей'
    
data['child_exist'] = data.apply(child_ex, axis=1)

In [49]:
#сгруппируем данные по столбцу 'child_exist' и вычислим необходимые значения столбца с наличием задолженности по кредиту
child_depend = data.groupby('child_exist').agg({'debt':['sum', 'count']})
child_depend

debt       
               sum  count
child_exist              
есть ребенок   669   7240
нет детей     1072  14214

In [50]:
#вычислим отношение должников к общему числу людей

child_depend_rel = child_depend['debt']['sum']/child_depend['debt']['count']
child_depend_rel.sort_values(ascending=False)

child_exist
есть ребенок    0.092403
нет детей       0.075419
dtype: float64

<div style="border: 15px; border: 2px solid blue; padding: 15px;">
<b> Комментарий студента:</b>
<hr>
    Далее ответим на вопрос есть ли зависимость между семейным положением и возвратом кредита в срок? Интересно узнать как влияет наличие партнера на споособность возврата кредита в срок. 
</div>

In [51]:
#сгруппируем данные по столбцу 'fam_partner' и вычислим необходимые значения столбца с наличием задолженности по кредиту

fam_stat_depend = data.groupby('family_status').agg({'debt':['sum', 'count']})
fam_stat_depend

debt       
                       sum  count
family_status                    
Не женат / не замужем  274   2810
в разводе               85   1195
вдовец / вдова          63    959
гражданский брак       388   4151
женат / замужем        931  12339

In [52]:
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmatized_purpose,child_exist
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",есть ребенок
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",есть ребенок
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",нет детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",есть ребенок
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",нет детей
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,"[операция, , с, , жилье, \n]",есть ребенок
21450,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,"[сделка, , с, , автомобиль, \n]",нет детей
21451,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,"[недвижимость, \n]",есть ребенок
21452,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,"[на, , покупка, , свой, , автомобиль, \n]",есть ребенок


In [53]:
# посчитаем отношение должников с партнером и без к общему числу людей
fam_stat_depend_rel = fam_stat_depend['debt']['sum']/fam_stat_depend['debt']['count']
fam_stat_depend_rel.sort_values(ascending=False)

family_status
Не женат / не замужем    0.097509
гражданский брак         0.093471
женат / замужем          0.075452
в разводе                0.071130
вдовец / вдова           0.065693
dtype: float64

<div style="border: 15px; border: 2px solid blue; padding: 15px;">
<b> Комментарий студента:</b>
<hr>
    Для того, чтобы ответить на вопрос есть ли зависимость между уровнем дохода и возвратом кредита в срок, необходимо категоризировать данные из стоблца 'total_income'.
</div>

In [54]:
#для начала определим граничные и среднее значения столбца 'total_income'

income_index = data['total_income'].agg({'total_income':['min', 'max', 'mean']})

In [55]:
#далее напишем функцию, которая бы позволила присвоить уровням дохода категории

def category_income(row):
    
    total_income = row['total_income']
    
    if total_income <= (income_index['total_income']['min']+income_index['total_income']['mean'])/2:
        return 'низкий'
    
    if (income_index['total_income']['min']+income_index['total_income']['mean'])/2 < total_income <= income_index['total_income']['mean']:
        return 'ниже среднего'
        
    if  income_index['total_income']['mean'] < total_income <= (income_index['total_income']['mean']+income_index['total_income']['max'])/2:
        return 'выше среднего'
    
    if  total_income > (income_index['total_income']['mean']+income_index['total_income']['max'])/2:
        return 'высокий'
    
data['category_income'] = data.apply(category_income, axis=1)

In [56]:
data['category_income'].value_counts().sum()

21454

In [57]:
income_depend = data.groupby('category_income').agg({'debt':['sum', 'count']})
income_depend

debt      
                 sum count
category_income           
высокий            1    13
выше среднего    623  8158
ниже среднего    826  9618
низкий           291  3665

In [58]:
income_depend_rel = income_depend['debt']['sum']/income_depend['debt']['count']
income_depend_rel.sort_values(ascending=False)

category_income
ниже среднего    0.085881
низкий           0.079400
высокий          0.076923
выше среднего    0.076367
dtype: float64

**Вывод**

В данном разделе была проведена категоризация данных по следующим признакам: по наличию детей, по семейному положению, а также по ежемесячному доходу. 

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [59]:
child_depend_rel.sort_values(ascending=False)

child_exist
есть ребенок    0.092403
нет детей       0.075419
dtype: float64

**Вывод**

Существует зависимость между наличием либо отсутсвием детей и возвратом кредита в срок. Люди, у которых нет детей, имели наименьший процент просрочек по сравнению с теми, у кого есть дети.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [60]:
fam_stat_depend_rel.sort_values(ascending=False)

family_status
Не женат / не замужем    0.097509
гражданский брак         0.093471
женат / замужем          0.075452
в разводе                0.071130
вдовец / вдова           0.065693
dtype: float64

In [61]:
data.pivot_table(index='family_status', columns='debt', values='family_status_id', aggfunc='count')

debt,0,1
family_status,,
Не женат / не замужем,2536,274
в разводе,1110,85
вдовец / вдова,896,63
гражданский брак,3763,388
женат / замужем,11408,931


**Вывод**

Как можно заметить, существует зависимость между семейным положением и возвратом кредита в срок. Наличие партнера положительно влияет на способность человека выплачивать кредит в срок. Это связано с тем, что в трудной ситуации есть возможность обратиться за помощью к партнеру.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [62]:
income_depend_rel.sort_values(ascending=False)

category_income
ниже среднего    0.085881
низкий           0.079400
высокий          0.076923
выше среднего    0.076367
dtype: float64

**Вывод**

По данным, предствленным выше, можно сказать о том, что существует явная зависимость между уровнем дохода и возвратом кредита в срок. Люди с низким доходом имеют больше просрочек по кредиту, чем люди с доходом выше среднего. 

- Как разные цели кредита влияют на его возврат в срок?

In [63]:
purpose_debt_avg.sort_values(ascending=False)

lemmatized_purpose
автомобиль      0.093590
образование     0.092200
свадьба         0.080034
недвижимость    0.072334
dtype: float64

**Вывод**

По результатам проведенной работы можно сказать, что наиболее часто имеют задолженность люди, которые берут кредит на автомобили и образование, это связано с тем, что ежемесячный доход у этих групп людей ниже среднего. При этом меньше всего людей имеют задолженность по кредиту на недвижимость, это объясняется более высоким ежемесячным доходом.

## Шаг 4. Общий вывод

Мы ответили на 4 вопроса и установили, что:

1. Существует зависимость между наличием либо отсутсвием детей и возвратом кредита в срок. Люди, у которых нет детей, имели наименьший процент просрочек по сравнению с теми, у кого есть дети.


2. Существует зависимость между семейным положением и возвратом кредита в срок. Наличие партнера положительно влияет на способность человека выплачивать кредит в срок. Это связано с тем, что в трудной ситуации есть возможность обратиться за помощью к партнеру.


3. Существует явная зависимость между уровнем дохода и возвратом кредита в срок. Люди с низким доходом имеют больше просрочек по кредиту, чем люди с доходом выше среднего. 


4. По результатам проведенной работы можно сказать, что наиболее часто имеют задолженность люди, которые берут кредит на автомобили и образование, это связано с тем, что ежемесячный доход у этих групп людей ниже среднего. При этом меньше всего людей имеют задолженность по кредиту на недвижимость, это объясняется более высоким ежемесячным доходом.